In [1]:
import pandas as pd
import numpy as np
import collections
import re
from collections import Counter
from collections import namedtuple
import os
import pickle
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import time
import sys
import random
import nltk

In [2]:
train_data = pd.read_csv("../Data/en_train.csv")
test_data = pd.read_csv("../Data/en_test_2.csv")
sample_submission = pd.read_csv("../Data/en_sample_submission_2.csv")
train_data.before = train_data.before.astype(str)
train_data.after = train_data.after.astype(str)
test_data.before = test_data.before.astype(str)
train_data['prev_before'] = train_data['before'].shift(1)
train_data['prev_before'][train_data.sentence_id != train_data.sentence_id.shift(1)] = ""
train_data['prev_prev_before'] = train_data['before'].shift(2)
train_data['prev_prev_before'][train_data.sentence_id != train_data.sentence_id.shift(2)] = ""
train_data['next_before'] = train_data['before'].shift(-1)
train_data['next_before'][train_data.sentence_id != train_data.sentence_id.shift(-1)] = ""
train_data['next_next_before'] = train_data['before'].shift(-2)
train_data['next_next_before'][train_data.sentence_id != train_data.sentence_id.shift(-2)] = ""

train_data['before_phrase'] = (train_data.prev_prev_before + "<SEP>" + train_data.prev_before + "<SEP>" +  train_data.before 
                                + "<SEP>" + train_data.next_before + "<SEP>" + train_data.next_next_before)
def truncate(x, phrase_len = 10):
    words = x.split("<SEP>")
    truncated_words = []
    if len(words[1]) < phrase_len:
        if len(words[0]) < phrase_len - len(words[1]):
            truncated_words.append(words[0])
            truncated_words.append(words[1])
        else:
            truncated_words.append(words[0][-(phrase_len - len(words[1])) : ])
            truncated_words.append(words[1])
    else:
        truncated_words.append("")
        truncated_words.append(words[1][-phrase_len : ])
    truncated_words.append(words[2])
    if len(words[3]) < phrase_len:
        if len(words[4]) < phrase_len - len(words[3]):
            truncated_words.append(words[3])
            truncated_words.append(words[4])
        else:
            truncated_words.append(words[3])
            truncated_words.append(words[4][-(phrase_len - len(words[3])) : ])
    else:
        truncated_words.append(words[3][-phrase_len : ])
        truncated_words.append("")
    return "<SEP>".join(truncated_words)
train_data['before_phrase'] = train_data['before_phrase'].map(truncate)
print train_data.shape, test_data.shape
random.seed(5)
dev_sentences = set(random.sample(list(set(train_data.sentence_id)),int(0.8* train_data.sentence_id.nunique())))
dev_data = train_data[train_data.sentence_id.isin(dev_sentences)]
val_data = train_data[train_data.sentence_id.isin(dev_sentences) == False]
print dev_data.shape, val_data.shape

print train_data['class'].value_counts()
print dev_data['class'].value_counts()
print val_data['class'].value_counts()

/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/sourabhjha/anaconda2/lib/pyth

(9918441, 10) (956046, 3)
(7935384, 10) (1983057, 10)
PLAIN         7353693
PUNCT         1880507
DATE           258348
LETTERS        152795
CARDINAL       133744
VERBATIM        78108
MEASURE         14783
ORDINAL         12703
DECIMAL          9821
MONEY            6128
DIGIT            5442
ELECTRONIC       5162
TELEPHONE        4024
TIME             1465
FRACTION         1196
ADDRESS           522
Name: class, dtype: int64
PLAIN         5882948
PUNCT         1505157
DATE           206643
LETTERS        122165
CARDINAL       106879
VERBATIM        62691
MEASURE         11768
ORDINAL         10090
DECIMAL          7905
MONEY            4918
DIGIT            4363
ELECTRONIC       4156
TELEPHONE        3186
TIME             1182
FRACTION          917
ADDRESS           416
Name: class, dtype: int64
PLAIN         1470745
PUNCT          375350
DATE            51705
LETTERS         30630
CARDINAL        26865
VERBATIM        15417
MEASURE          3015
ORDINAL          2613
DECIMAL       

In [3]:
rx = re.compile("[A-Za-z0-9]\.[a-z]")
dev_data_numeric = dev_data[(dev_data.before.map(lambda x : max([i in x for i in "0123456789"])) == 1) & 
                       (dev_data.before.map(lambda x : 1.0 if len(rx.findall(x)) > 0 and 
                        " " not in x and len(x) > 5 else 0.0) == 0.0)]
val_data_numeric = val_data[(val_data.before.map(lambda x : max([i in x for i in "0123456789"])) == 1) & 
                       (val_data.before.map(lambda x : 1.0 if len(rx.findall(x)) > 0 and 
                        " " not in x and len(x) > 5 else 0.0) == 0.0)]
dev_data_numeric['class'].value_counts()

DATE          206643
CARDINAL      103386
MEASURE        11689
ORDINAL         8843
DECIMAL         7905
MONEY           4918
DIGIT           4363
TELEPHONE       3186
TIME            1182
FRACTION         907
ADDRESS          416
VERBATIM         226
ELECTRONIC        37
PLAIN             18
PUNCT              2
Name: class, dtype: int64

In [4]:
dev_data_letters = dev_data[(dev_data.index.isin(dev_data_numeric.index) == False) & 
                ((dev_data.before.map(lambda x : x.upper() == x)) | 
                 (dev_data.before.map(lambda x : max([i in "aeiouy" for i in x.lower()])) == 0)) &
                (dev_data.before.map(lambda x : min([i in "abcdefghijklmnopqrstuvwxyz. " for i in x.lower()])) == 1) &
                (dev_data.before.map(len) > 1)]

val_data_letters = val_data[(val_data.index.isin(val_data_numeric.index) == False) & 
                ((val_data.before.map(lambda x : x.upper() == x)) | 
                 (val_data.before.map(lambda x : max([i in "aeiouy" for i in x.lower()])) == 0)) &
                (val_data.before.map(lambda x : min([i in "abcdefghijklmnopqrstuvwxyz. " for i in x.lower()])) == 1) &
                (val_data.before.map(len) > 1)]

dev_data_letters['class'].value_counts()

LETTERS       99718
PLAIN         31670
VERBATIM       6887
CARDINAL       3015
ORDINAL         935
ELECTRONIC       25
Name: class, dtype: int64

In [124]:
dev_rest = dev_data[(dev_data.index.isin(dev_data_numeric.index) == False) & 
         (dev_data.index.isin(dev_data_letters.index) == False)]
val_rest = val_data[(val_data.index.isin(val_data_numeric.index) == False) & 
         (val_data.index.isin(val_data_letters.index) == False)]
punctuations = set(train_data[train_data['class'] =='PUNCT']['before']) - {"-", ":"}
val_punct = val_rest[val_rest.before.isin(punctuations)]
val_rest = val_rest[val_rest.before.isin(punctuations) == False]
print val_rest.shape, np.mean(val_rest.before == val_rest.after), val_punct.shape, np.mean(val_punct.before == val_punct.after)

dev_punct = dev_rest[dev_rest.before.isin(punctuations)]
dev_rest = dev_rest[dev_rest.before.isin(punctuations) == False]
print dev_rest.shape, np.mean(dev_rest.before == dev_rest.after), dev_punct.shape,np.mean(dev_punct.before == dev_punct.after)

clean_words = set(word.lower() for word in nltk.corpus.treebank.words()).union(
                                set(word.lower() for word in nltk.corpus.brown.words()))
clean_words.remove("-")
clean_words.remove(":")
clean_words.remove("i")
clean_words.remove("no")

print len(clean_words)
val_clean = val_rest[val_rest.before.map(lambda x : x.lower() in clean_words)]
val_rest = val_rest[val_rest.before.map(lambda x : x.lower() in clean_words) == False]
print val_rest.shape, np.mean(val_rest.before == val_rest.after), val_clean.shape, np.mean(val_clean.before == val_clean.after)

dev_clean = dev_rest[dev_rest.before.map(lambda x : x.lower() in clean_words)]
dev_rest = dev_rest[dev_rest.before.map(lambda x : x.lower() in clean_words) == False]
print dev_rest.shape, np.mean(dev_rest.before == dev_rest.after), dev_clean.shape, np.mean(dev_clean.before == dev_clean.after)

def clean_elec(x):
    return " ".join(x).replace(".", "dot").replace(":", "colon").replace("/", "s l a s h")
rel = re.compile("[A-Za-z]\.[a-z]+")
val_elec = val_rest[val_rest.before.map(lambda x : len(rel.findall(x)) > 0)]
val_rest = val_rest[val_rest.before.map(lambda x : len(rel.findall(x)) == 0)]
print val_rest.shape, np.mean(val_rest.before == val_rest.after),val_elec.shape, np.mean(val_elec.after != val_elec.before.map(clean_elec))

dev_elec = dev_rest[dev_rest.before.map(lambda x : len(rel.findall(x)) > 0)]
dev_rest = dev_rest[dev_rest.before.map(lambda x : len(rel.findall(x)) == 0)]
print dev_rest.shape, np.mean(dev_rest.before == dev_rest.after),dev_elec.shape, np.mean(dev_elec.after != dev_elec.before.map(clean_elec))


rpn = re.compile("[A-Z][a-z]+")
val_proper_nouns = val_rest[val_rest.before.map(lambda x : len(rpn.findall(x)) > 0)]
val_rest = val_rest[val_rest.before.map(lambda x : len(rpn.findall(x)) == 0)]
print val_rest.shape, np.mean(val_rest.before == val_rest.after), val_proper_nouns.shape, np.mean(val_proper_nouns.before == val_proper_nouns.after)

dev_proper_nouns = dev_rest[dev_rest.before.map(lambda x : len(rpn.findall(x)) > 0)]
dev_rest = dev_rest[dev_rest.before.map(lambda x : len(rpn.findall(x)) == 0)]
print dev_rest.shape, np.mean(dev_rest.before == dev_rest.after), dev_proper_nouns.shape, np.mean(dev_proper_nouns.before == dev_proper_nouns.after)


(1496295, 10) 0.990732442466 (362398, 10) 1.0
(5987031, 10) 0.990733303369 (1452382, 10) 0.999999311476
53228
(197767, 10) 0.949137115899 (1298528, 10) 0.99706744868
(793548, 10) 0.949560707103 (5193483, 10) 0.99702434763
(196846, 10) 0.953577923859 (921, 10) 0.757871878393
(789742, 10) 0.954136920665 (3806, 10) 0.786915396742
(69215, 10) 0.892942281297 (127631, 10) 0.986460969514
(278029, 10) 0.895118854508 (511713, 10) 0.986203203749


In [6]:
try:
    chars_numeric = pickle.load(open("./chars_numeric.pkl", 'rb'))
except IOError:
    chars_numeric = set(c for token in list(dev_data_numeric.before_phrase.map(lambda x : x.replace("<SEP>",""))) + 
                list(dev_data_numeric.after) for c in token)
    chars_numeric = ['<PAD>','<EOS>','<GO>','<SELF>', '<SEP>'] + list(chars_numeric)
    pickle.dump(chars_numeric, open("./chars_numeric.pkl","wb"))
char_to_int_numeric = {c : i for i,c in enumerate(chars_numeric)}
int_to_char_numeric = {i : c for i,c in enumerate(chars_numeric)}
len(chars_numeric)

183

In [7]:
try:
    chars_letters = pickle.load(open("./chars_letters.pkl", 'rb'))
except IOError:
    chars_letters = set(c for token in list(dev_data_letters.before_phrase.map(lambda x : x.replace("<SEP>",""))) + 
                list(dev_data_letters.after) for c in token)
    chars_letters = ['<PAD>','<EOS>','<GO>','<SELF>', '<SEP>'] + list(chars_letters)
    pickle.dump(chars_letters, open("./chars_letters.pkl","wb"))
char_to_int_letters = {c : i for i,c in enumerate(chars_letters)}
int_to_char_letters = {i : c for i,c in enumerate(chars_letters)}
len(set(chars_letters))

178

In [8]:
try:
    chars_rest = pickle.load(open("./chars_rest.pkl", 'rb'))
except IOError:
    chars_rest = set(c for token in list(dev_rest.before_phrase.map(lambda x : x.replace("<SEP>",""))) + 
                list(dev_rest.after) for c in token)
    chars_rest = ['<PAD>','<EOS>','<GO>','<SELF>', '<SEP>'] + list(chars_rest)
    pickle.dump(chars_rest, open("./chars_rest.pkl","wb"))
char_to_int_rest = {c : i for i,c in enumerate(chars_rest)}
int_to_char_rest = {i : c for i,c in enumerate(chars_rest)}
len(chars_rest)

192

In [9]:
try:
    chars_proper_nouns = pickle.load(open("./chars_proper_nouns.pkl", 'rb'))
except IOError:
    chars_proper_nouns = set(c for token in list(dev_proper_nouns.before_phrase.map(lambda x : x.replace("<SEP>",""))) + 
                list(dev_proper_nouns.after) for c in token)
    chars_proper_nouns = ['<PAD>','<EOS>','<GO>','<SELF>', '<SEP>'] + list(chars_proper_nouns)
    pickle.dump(chars_proper_nouns, open("./chars_proper_nouns.pkl","wb"))
char_to_int_proper_nouns = {c : i for i,c in enumerate(chars_proper_nouns)}
int_to_char_proper_nouns = {i : c for i,c in enumerate(chars_proper_nouns)}
len(chars_proper_nouns)

188

In [142]:
def get_char_to_int_numeric(c):
    try:
        return char_to_int_numeric[c]
    except KeyError:
        return char_to_int_numeric[" "]
def get_char_to_int_letters(c):
    try:
        return char_to_int_letters[c]
    except KeyError:
        return char_to_int_letters[" "]

def get_char_to_int_proper_nouns(c):
    try:
        return char_to_int_proper_nouns[c]
    except KeyError:
        return char_to_int_proper_nouns[" "]

def get_char_to_int_rest(c):
    try:
        return char_to_int_rest[c]
    except KeyError:
        return char_to_int_rest[" "]
    
int_before_dev_numeric = []
int_after_dev_numeric = []
int_before_val_numeric = []
int_after_val_numeric = []

int_before_dev_letters = []
int_after_dev_letters = []
int_before_val_letters = []
int_after_val_letters = []

int_before_dev_rest = []
int_after_dev_rest = []
int_before_val_rest = []
int_after_val_rest = []

int_before_dev_proper_nouns = []
int_after_dev_proper_nouns = []
int_before_val_proper_nouns = []
int_after_val_proper_nouns = []

dev_words_numeric = dev_data_numeric[['before_phrase', 'after']].drop_duplicates()
for before, after in zip(dev_words_numeric.before_phrase, dev_words_numeric.after):
    words = before.split("<SEP>")
    if words[2] == after:
        int_before_dev_numeric.append([get_char_to_int_numeric(c) for c in words[0]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[1]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[2]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[3]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[4]])
        int_after_dev_numeric.append([get_char_to_int_numeric('<SELF>')])
    else:
        int_before_dev_numeric.append([get_char_to_int_numeric(c) for c in words[0]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[1]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[2]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[3]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[4]])
        int_after_dev_numeric.append([get_char_to_int_numeric(c) for c in after])

        
val_words_numeric = val_data_numeric[['before_phrase', 'after']].drop_duplicates()
for before, after in zip(val_words_numeric.before_phrase, val_words_numeric.after):
    words = before.split("<SEP>")
    if words[2] == after:
        int_before_val_numeric.append([get_char_to_int_numeric(c) for c in words[0]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[1]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[2]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[3]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[4]])
        int_after_val_numeric.append([get_char_to_int_numeric('<SELF>')])
    else:
        int_before_val_numeric.append([get_char_to_int_numeric(c) for c in words[0]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[1]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[2]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[3]] + [char_to_int_numeric['<SEP>']] 
                                    + [get_char_to_int_numeric(c) for c in words[4]])
        int_after_val_numeric.append([get_char_to_int_numeric(c) for c in after])

dev_words_letters = dev_data_letters[['before_phrase', 'after']].drop_duplicates()
for before, after in zip(dev_words_letters.before_phrase, dev_words_letters.after):
    words = before.split("<SEP>")
    if words[2] == after:
        int_before_dev_letters.append([get_char_to_int_letters(c) for c in words[0]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[1]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[2]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[3]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[4]])
        int_after_dev_letters.append([get_char_to_int_letters('<SELF>')])
    else:
        int_before_dev_letters.append([get_char_to_int_letters(c) for c in words[0]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[1]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[2]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[3]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[4]])
        int_after_dev_letters.append([get_char_to_int_letters(c) for c in after])

        
val_words_letters = val_data_letters[['before_phrase', 'after']].drop_duplicates()
for before, after in zip(val_words_letters.before_phrase, val_words_letters.after):
    words = before.split("<SEP>")
    if words[2] == after:
        int_before_val_letters.append([get_char_to_int_letters(c) for c in words[0]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[1]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[2]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[3]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[4]])
        int_after_val_letters.append([get_char_to_int_letters('<SELF>')])
    else:
        int_before_val_letters.append([get_char_to_int_letters(c) for c in words[0]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[1]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[2]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[3]] + [char_to_int_letters['<SEP>']] 
                                    + [get_char_to_int_letters(c) for c in words[4]])
        int_after_val_letters.append([get_char_to_int_letters(c) for c in after])

dev_words_rest = dev_rest[['before_phrase', 'after']].drop_duplicates()
for before, after in zip(dev_words_rest.before_phrase, dev_words_rest.after):
    words = before.split("<SEP>")
    if words[2] == after:
        int_before_dev_rest.append([get_char_to_int_rest(c) for c in words[0]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[1]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[2]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[3]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[4]])
        int_after_dev_rest.append([get_char_to_int_rest('<SELF>')])
    else:
        int_before_dev_rest.append([get_char_to_int_rest(c) for c in words[0]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[1]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[2]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[3]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[4]])
        int_after_dev_rest.append([get_char_to_int_rest(c) for c in after])

        
val_words_rest = val_rest[['before_phrase', 'after']].drop_duplicates()
for before, after in zip(val_words_rest.before_phrase, val_words_rest.after):
    words = before.split("<SEP>")
    if words[2] == after:
        int_before_val_rest.append([get_char_to_int_rest(c) for c in words[0]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[1]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[2]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[3]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[4]])
        int_after_val_rest.append([get_char_to_int_rest('<SELF>')])
    else:
        int_before_val_rest.append([get_char_to_int_rest(c) for c in words[0]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[1]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[2]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[3]] + [char_to_int_rest['<SEP>']] 
                                    + [get_char_to_int_rest(c) for c in words[4]])
        int_after_val_rest.append([get_char_to_int_rest(c) for c in after])
        
dev_words_proper_nouns = dev_proper_nouns[['before_phrase', 'after']].drop_duplicates()
for before, after in zip(dev_words_proper_nouns.before_phrase, dev_words_proper_nouns.after):
    words = before.split("<SEP>")
    if words[2] == after:
        int_before_dev_proper_nouns.append([get_char_to_int_proper_nouns(c) for c in words[0]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[1]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[2]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[3]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[4]])
        int_after_dev_proper_nouns.append([get_char_to_int_proper_nouns('<SELF>')])
    else:
        int_before_dev_proper_nouns.append([get_char_to_int_proper_nouns(c) for c in words[0]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[1]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[2]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[3]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[4]])
        int_after_dev_proper_nouns.append([get_char_to_int_proper_nouns(c) for c in after])

        
val_words_proper_nouns = val_proper_nouns[['before_phrase', 'after']].drop_duplicates()
for before, after in zip(val_words_proper_nouns.before_phrase, val_words_proper_nouns.after):
    words = before.split("<SEP>")
    if words[2] == after:
        int_before_val_proper_nouns.append([get_char_to_int_proper_nouns(c) for c in words[0]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[1]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[2]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[3]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[4]])
        int_after_val_proper_nouns.append([get_char_to_int_proper_nouns('<SELF>')])
    else:
        int_before_val_proper_nouns.append([get_char_to_int_proper_nouns(c) for c in words[0]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[1]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[2]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[3]] + [char_to_int_proper_nouns['<SEP>']] 
                                    + [get_char_to_int_proper_nouns(c) for c in words[4]])
        int_after_val_proper_nouns.append([get_char_to_int_proper_nouns(c) for c in after])
        
print len(int_before_dev_numeric), len(int_after_dev_numeric), len(int_before_val_numeric), len(int_after_val_numeric)
print len(int_before_dev_letters), len(int_after_dev_letters), len(int_before_val_letters), len(int_after_val_letters)
print len(int_before_dev_rest), len(int_after_dev_rest), len(int_before_val_rest), len(int_after_val_rest)
print len(int_before_dev_rest), len(int_after_dev_rest), len(int_before_val_rest), len(int_after_val_rest)

325360 325360 84776 84776
130746 130746 33434 33434
261277 261277 66917 66917
261277 261277 66917 66917


In [11]:
def model_inputs():
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [None, None], name='targets')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    inputs_length = tf.placeholder(tf.int32, (None,), name = 'inputs_length')
    targets_length = tf.placeholder(tf.int32, (None,), name = 'targets_length')
    max_targets_length = tf.reduce_max(targets_length, name = 'max_target_len')
    
    return inputs, targets, keep_prob, inputs_length, targets_length, max_targets_length

In [12]:
def process_encoding_input(targets, vocab_to_int, batch_size):
    with tf.name_scope("process_encoding"):
        ending = tf.strided_slice(targets, [0,0], [batch_size, - 1], [1,1])
        dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending],1)
    return dec_input

def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob, direction):
    if direction == 1:
        with tf.name_scope("RNN_Encoder_Cell_1D"):
            for layer in range(num_layers):
                with tf.variable_scope('encoder_{}'.format(layer)):
                    lstm = tf.contrib.rnn.LSTMCell(rnn_size)
                    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
                    enc_output, enc_state = tf.nn.dynamic_rnn(drop,rnn_inputs,sequence_length, dtype=tf.float32)
            return enc_output, enc_state
    if direction ==2:
        with tf.name_scope("RNN_Encoder_Cell_2D"):
            for layer in range(num_layers):
                with tf.variable_scope('encoder_{}'.format(layer)):
                    cell_fw = tf.contrib.rnn.LSTMCell(rnn_size)
                    cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob = keep_prob)
                    cell_bw = tf.contrib.rnn.LSTMCell(rnn_size)
                    cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob = keep_prob)
                    
                    enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw,
                                                                            rnn_inputs,sequence_length, 
                                                                            dtype=tf.float32)
            enc_output = tf.concat(enc_output,2)
            return enc_output, enc_state[0]
        
def training_decoding_layer(dec_embed_input, targets_length, dec_cell, initial_state, output_layer,vocab_size,
                           max_target_length):
    with tf.name_scope("Training_Decoder"):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs = dec_embed_input, sequence_length = targets_length,
                                                           time_major = False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, initial_state, output_layer)
        training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder, output_time_major = False,
                                                              impute_finished = True,
                                                              maximum_iterations = max_target_length)
        return training_logits

def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer, 
                             max_target_length, batch_size):
    with tf.name_scope("Inference_Decoder"):
        start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings, start_tokens, end_token)
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, inference_helper, initial_state, output_layer)
        inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder, output_time_major=False,
                                                               impute_finished = True,
                                                               maximum_iterations = max_target_length)
        return inference_logits

In [13]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, inputs_length, targets_length, 
                   max_target_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers, direction):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    with tf.name_scope("RNN_Decoder_Cell"):
        for layer in range(num_layers):
            with tf.variable_scope('decoder_{}'.format(layer)):
                lstm = tf.contrib.rnn.LSTMCell(rnn_size)
                dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size, enc_output,inputs_length,normalize=False,
                                                                                  name='BahdanauAttention')
    
    with tf.name_scope("Attention_Wrapper"):
        dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,attn_mech, rnn_size)
    
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state,_zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32))

    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, targets_length, dec_cell, initial_state,
                                                  output_layer,vocab_size, max_target_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings, vocab_to_int['<GO>'], vocab_to_int['<EOS>'],
                                                    dec_cell, initial_state, output_layer,
                                                    max_target_length,batch_size)
    return training_logits, inference_logits

In [14]:
def seq2seq_model(inputs, targets, keep_prob, inputs_length, targets_length, max_target_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size, embedding_size, direction):
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1, 1))
    enc_embed_input = tf.nn.embedding_lookup(enc_embeddings, inputs)
    enc_output, enc_state = encoding_layer(rnn_size, inputs_length, num_layers, 
                                           enc_embed_input, keep_prob, direction)
    
    dec_embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1, 1))
    dec_input = process_encoding_input(targets, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, dec_embeddings,enc_output,enc_state, 
                                                        vocab_size, inputs_length, targets_length, max_target_length,
                                                        rnn_size, vocab_to_int, keep_prob, batch_size,num_layers,
                                                        direction) 
    return training_logits, inference_logits

In [15]:
def pad_sentence_batch(sentence_batch, char_to_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [char_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]
def get_batches(before_tokens, after_tokens, batch_size, char_to_int): 
    for batch_i in range(0, len(before_tokens)//batch_size):
        start_i = batch_i * batch_size
        before_tokens_batch = before_tokens[start_i:start_i + batch_size]
        after_tokens_batch = after_tokens[start_i:start_i + batch_size]
            
        after_tokens_batch_eos = []
        for token in after_tokens_batch:
            token.append(char_to_int['<EOS>'])
            after_tokens_batch_eos.append(token)
            
        pad_after_tokens_batch = np.array(pad_sentence_batch(after_tokens_batch_eos, char_to_int))
        pad_before_tokens_batch = np.array(pad_sentence_batch(before_tokens_batch, char_to_int))
        
        # Need the lengths for the _lengths parameters
        pad_before_tokens_lengths = []
        for token in pad_before_tokens_batch:
            pad_before_tokens_lengths.append(len(token))
        
        pad_after_tokens_lengths = []
        for token in pad_after_tokens_batch:
            pad_after_tokens_lengths.append(len(token))
        
        yield pad_before_tokens_batch, pad_after_tokens_batch, pad_before_tokens_lengths, pad_after_tokens_lengths

In [16]:
def build_graph(keep_prob, rnn_size, num_layers, batch_size, learning_rate, embedding_size, direction, char_to_int):

    tf.reset_default_graph()
    
    # Load the model inputs    
    inputs, targets, keep_prob, inputs_length, targets_length, max_target_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(inputs, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      inputs_length,
                                                      targets_length,
                                                      max_target_length,
                                                      len(char_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      char_to_int,
                                                      batch_size,
                                                      embedding_size,
                                                      direction)

    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')

    with tf.name_scope('predictions'):
        predictions = tf.identity(inference_logits.sample_id, name='predictions')
        tf.summary.histogram('predictions', predictions)

    # Create the weights for sequence_loss
    masks = tf.sequence_mask(targets_length, max_target_length, dtype=tf.float32, name='masks')
    
    with tf.name_scope("cost"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, 
                                                targets, 
                                                masks)
        tf.summary.scalar('cost', cost)

    with tf.name_scope("optimze"):
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

    # Merge all of the summaries
    merged = tf.summary.merge_all()    

    # Export the nodes 
    export_nodes = ['inputs', 'targets', 'keep_prob', 'cost', 'inputs_length', 'targets_length',
                    'predictions', 'merged', 'train_op','optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])

    return graph


# ## Training the Model

# In[74]:

def train(model, epochs, log_string, int_before_train, int_after_train, int_before_test, int_after_test, char_to_int,
         reset = True, early_stopping = 3):
    '''Train the RNN'''
    
    with tf.Session() as sess:
        if reset:
            sess.run(tf.global_variables_initializer())
        else:
            try:
                checkpoint = "./{}.ckpt".format(log_string)
                saver = tf.train.Saver()
                saver.restore(sess, checkpoint)
            except tf.errors.NotFoundError:
                print "checkpoint not found..."
                print "initiating new model..."
                sess.run(tf.global_variables_initializer())
        # Used to determine when to stop the training early
        testing_loss_summary = []

        # Keep track of which batch iteration is being trained
        iteration = 0
        
        display_step = 50 # The progress of the training will be displayed after every 30 batches
        stop_early = 0 
        stop = early_stopping # If the batch_loss_testing does not decrease in 3 consecutive checks, stop training
        per_epoch = 1 # Test the model 3 times per epoch
        testing_check = (len(int_before_train)//batch_size//per_epoch)-1

        print()
        print("Training Model: {}".format(log_string))

        train_writer = tf.summary.FileWriter('./logs/1/train/{}'.format(log_string), sess.graph)
        test_writer = tf.summary.FileWriter('./logs/1/test/{}'.format(log_string))

        for epoch_i in range(1, epochs+1): 
            batch_loss = 0
            batch_time = 0
            
            for batch_i, (input_batch, target_batch, input_length, target_length) in enumerate(
                    get_batches(int_before_train, int_after_train, batch_size, char_to_int)):
                start_time = time.time()

                summary, loss, _ = sess.run([model.merged,
                                             model.cost, 
                                             model.train_op], 
                                             {model.inputs: input_batch,
                                              model.targets: target_batch,
                                              model.inputs_length: input_length,
                                              model.targets_length: target_length,
                                              model.keep_prob: keep_probability})


                batch_loss += loss
                end_time = time.time()
                batch_time += end_time - start_time
#                 sys.stdout.write(str(batch_i) + ".")
                # Record the progress of training
                train_writer.add_summary(summary, iteration)

                iteration += 1

                if batch_i % display_step == 0 and batch_i > 0:
                    print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                          .format(epoch_i,
                                  epochs, 
                                  batch_i, 
                                  len(int_before_train) // batch_size, 
                                  batch_loss / display_step, 
                                  batch_time))
                    batch_loss = 0
                    batch_time = 0

                #### Testing ####
                if batch_i % testing_check == 0 and batch_i > 0:
                    batch_loss_testing = 0
                    batch_time_testing = 0
                    for batch_i, (input_batch, target_batch, input_length, target_length) in enumerate(
                            get_batches(int_before_test, int_after_test, batch_size,char_to_int)):
                        start_time_testing = time.time()
                        summary, loss = sess.run([model.merged,
                                                  model.cost], 
                                                     {model.inputs: input_batch,
                                                      model.targets: target_batch,
                                                      model.inputs_length: input_length,
                                                      model.targets_length: target_length,
                                                      model.keep_prob: 1})

                        batch_loss_testing += loss
                        end_time_testing = time.time()
                        batch_time_testing += end_time_testing - start_time_testing

                        # Record the progress of testing
                        test_writer.add_summary(summary, iteration)

                    n_batches_testing = batch_i + 1
                    print('Testing Loss: {:>6.3f}, Seconds: {:>4.2f}'
                          .format(batch_loss_testing / n_batches_testing, 
                                  batch_time_testing))
                    
                    batch_time_testing = 0

                    # If the batch_loss_testing is at a new minimum, save the model
                    testing_loss_summary.append(batch_loss_testing)
                    if batch_loss_testing <= min(testing_loss_summary):
                        print('New Record!') 
                        stop_early = 0
                        checkpoint = "./{}.ckpt".format(log_string)
                        saver = tf.train.Saver()
                        saver.save(sess, checkpoint)

                    else:
                        print("No Improvement.")
                        stop_early += 1
                        if stop_early == stop:
                            break
            sys.stdout.flush()
            if stop_early == stop:
                print("Stopping Training.")
                break

In [143]:
epochs = 100
batch_size = 500
num_layers = 3
rnn_size = 128
embedding_size = len(chars_numeric)
learning_rate = 0.0005
direction = 2
keep_probability = 0.75

log_string = 'all_numeric_model_1'
model_numeric = build_graph(keep_probability, rnn_size, num_layers, batch_size, 
                    learning_rate, embedding_size, direction, char_to_int_numeric)
train(model_numeric, epochs, log_string, int_before_dev_numeric, int_after_dev_numeric, 
      int_before_val_numeric, int_after_val_numeric, char_to_int_numeric, reset = False)

INFO:tensorflow:Restoring parameters from ./all_numeric_model_1.ckpt
()
Training Model: all_numeric_model_1
Epoch   1/100 Batch   50/650 - Loss:  0.007, Seconds: 215.52
Epoch   1/100 Batch  100/650 - Loss:  0.001, Seconds: 230.97
Epoch   1/100 Batch  150/650 - Loss:  0.001, Seconds: 221.05
Epoch   1/100 Batch  200/650 - Loss:  0.001, Seconds: 228.79
Epoch   1/100 Batch  250/650 - Loss:  0.001, Seconds: 219.00
Epoch   1/100 Batch  300/650 - Loss:  0.001, Seconds: 243.66
Epoch   1/100 Batch  350/650 - Loss:  0.001, Seconds: 223.90
Epoch   1/100 Batch  400/650 - Loss:  0.001, Seconds: 234.34
Epoch   1/100 Batch  450/650 - Loss:  0.001, Seconds: 230.42
Epoch   1/100 Batch  500/650 - Loss:  0.001, Seconds: 218.54
Epoch   1/100 Batch  550/650 - Loss:  0.001, Seconds: 242.34
Epoch   1/100 Batch  600/650 - Loss:  0.001, Seconds: 240.95
Testing Loss:  0.001, Seconds: 371.77
New Record!
Epoch   2/100 Batch   50/650 - Loss:  0.013, Seconds: 218.14
Epoch   2/100 Batch  100/650 - Loss:  0.001, Seco

KeyboardInterrupt: 

In [17]:
def token_to_ints_numeric(token):
    return [char_to_int_numeric[c] for c in token if c in chars_numeric]


def score_all_numeric(phrases, max_target_length = 150):
    checkpoint = "./all_numeric_model_1.ckpt"
    epochs = 100
    batch_size = 500
    num_layers = 3
    rnn_size = 128
    embedding_size = len(chars_numeric)
    learning_rate = 0.0005
    direction = 2
    keep_probability = 0.75
#     tokens1 = [token_to_ints(prev_token) + [char_to_int['<SEP>']] + token_to_ints(token) 
#                for prev_token, token in zip(prev_tokens, tokens)] 
    tokens1 = [reduce(lambda a,b : a + b, [token_to_ints_numeric(p) + [char_to_int_numeric['<SEP>']] 
                                           for p in phrase.split("<SEP>")])[:-1] for phrase in phrases] 
    
    pad_tokens = pad_sentence_batch(tokens1, char_to_int_numeric)
        
        # Need the lengths for the _lengths parameters
    pad_tokens_lengths = []
    pad_targets_lengths = []
    for token in pad_tokens:
        pad_tokens_lengths.append(len(token))    
        pad_targets_lengths.append(len(token) + max_target_length)
    model = build_graph(keep_probability, rnn_size, num_layers, len(phrases), 
                                learning_rate, embedding_size, direction, char_to_int_numeric)
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, checkpoint)
        
        output_logits = sess.run(model.predictions, {model.inputs: pad_tokens,
                                                    model.inputs_length: pad_tokens_lengths,
                                                    model.targets_length: [max_target_length],
                                                    model.keep_prob: 1.0})
    pad = char_to_int_numeric['<PAD>']
    return ["".join([int_to_char_numeric[i] for i in output_logit if i != pad]).replace("<EOS>","").replace("<SELF>", phrases[j].split("<SEP>")[2])
            for j, output_logit in enumerate(output_logits)]


In [144]:
n_pred = 10000
predictions =  score_all_numeric(list(val_data_numeric['before_phrase'])[:n_pred])
res = pd.DataFrame(zip(list(val_data_numeric['before_phrase'])[:n_pred],list(val_data_numeric['before'])[:n_pred], 
                       list(val_data_numeric['after'])[:n_pred],predictions))

print(np.mean(res[2] == res[3]))
res[res[2] != res[3]]

INFO:tensorflow:Restoring parameters from ./all_numeric_model_1.ckpt
0.992


,0,1,2,3
109,<SEP>The<SEP>12639/12640<SEP>Brindavan<SEP>s,12639/12640,twelve thousand six hundred thirty nine twelve...,twelve thousand six hundred thirty nine one tw...
151,again<SEP>in<SEP>1907<SEP>-<SEP>1908,1907,one thousand nine hundred seven,nineteen o seven
152,1907<SEP>-<SEP>1908<SEP>.<SEP>,1908,one thousand nine hundred eight,nineteen o eight
547,bill<SEP>AB<SEP>1519<SEP>(<SEP>Ma,1519,one five one nine,fifteen nineteen
682,",<SEP>p<SEP>6955<SEP>-6960<SEP>lcolm",6955,six nine five five,six thousand nine hundred fifty five
695,(<SEP>ISBN<SEP>978-1-78076-355-2<SEP>)<SEP>Driver,978-1-78076-355-2,nine seven eight sil one sil seven eight o sev...,nine seven eight sil one sil one sil seven eig...
757,"""<SEP>BUILD<SEP>2015<SEP>News<SEP>:",2015,twenty fifteen,two o one five
957,(<SEP>ID-<SEP>3548<SEP>)<SEP>was,3548,three five four eight,three thousand five hundred forty eight
987,",<SEP>CM-<SEP>4206<SEP>and<SEP>CM-",4206,four two o six,four thousand two hundred six
1059,metres<SEP>(<SEP>45 yd<SEP>)<SEP>intervals,45 yd,forty five yards,forty five yeards


In [ ]:
n_pred = 5000
predictions =  score_all_numeric(list(dev_data_numeric['before_phrase'])[:n_pred])
res = pd.DataFrame(zip(list(dev_data_numeric['before_phrase'])[:n_pred],list(dev_data_numeric['before'])[:n_pred], 
                       list(dev_data_numeric['after'])[:n_pred],predictions))

print(np.mean(res[2] == res[3]))
res[res[2] != res[3]]

In [31]:
epochs = 100
batch_size = 400
num_layers = 3
rnn_size = 128
embedding_size = len(chars_letters)
learning_rate = 0.0003
direction = 2
keep_probability = 0.75

log_string = 'all_letters_model_1'
model_letters = build_graph(keep_probability, rnn_size, num_layers, batch_size, 
                    learning_rate, embedding_size, direction, char_to_int_letters)
train(model_letters, epochs, log_string, int_before_dev_letters, int_after_dev_letters, 
      int_before_val_letters, int_after_val_letters, char_to_int_letters, reset = False)

INFO:tensorflow:Restoring parameters from ./all_letters_model_1.ckpt
()
Training Model: all_letters_model_1
Epoch   1/100 Batch   50/326 - Loss:  0.321, Seconds: 52.89
Epoch   1/100 Batch  100/326 - Loss:  0.006, Seconds: 51.90
Epoch   1/100 Batch  150/326 - Loss:  0.005, Seconds: 51.09
Epoch   1/100 Batch  200/326 - Loss:  0.004, Seconds: 53.12
Epoch   1/100 Batch  250/326 - Loss:  0.004, Seconds: 52.66
Epoch   1/100 Batch  300/326 - Loss:  0.004, Seconds: 51.78
Testing Loss:  0.003, Seconds: 38.88
New Record!
Epoch   2/100 Batch   50/326 - Loss:  0.067, Seconds: 54.08
Epoch   2/100 Batch  100/326 - Loss:  0.006, Seconds: 53.21
Epoch   2/100 Batch  150/326 - Loss:  0.005, Seconds: 51.88
Epoch   2/100 Batch  200/326 - Loss:  0.004, Seconds: 54.39
Epoch   2/100 Batch  250/326 - Loss:  0.004, Seconds: 53.88
Epoch   2/100 Batch  300/326 - Loss:  0.004, Seconds: 53.20
Testing Loss:  0.003, Seconds: 39.59
New Record!
Epoch   3/100 Batch   50/326 - Loss:  0.042, Seconds: 55.56
Epoch   3/100 

KeyboardInterrupt: 

In [18]:
epochs = 100
batch_size = 500
num_layers = 3
rnn_size = 128
embedding_size = len(chars_letters)
learning_rate = 0.0005
direction = 2
keep_probability = 0.75

def token_to_ints_letters(token):
    return [char_to_int_letters[c] for c in token if c in chars_letters]


def score_all_letters(phrases, max_target_length = 150):
    checkpoint = "./all_letters_model_1.ckpt"
    epochs = 100
    batch_size = 500
    num_layers = 3
    rnn_size = 128
    embedding_size = len(chars_letters)
    learning_rate = 0.0005
    direction = 2
    keep_probability = 0.75
#     tokens1 = [token_to_ints(prev_token) + [char_to_int['<SEP>']] + token_to_ints(token) 
#                for prev_token, token in zip(prev_tokens, tokens)] 
    tokens1 = [reduce(lambda a,b : a + b, [token_to_ints_letters(p) + [char_to_int_letters['<SEP>']] 
                                           for p in phrase.split("<SEP>")])[:-1] for phrase in phrases] 
    
    pad_tokens = pad_sentence_batch(tokens1, char_to_int_letters)
        
        # Need the lengths for the _lengths parameters
    pad_tokens_lengths = []
    pad_targets_lengths = []
    for token in pad_tokens:
        pad_tokens_lengths.append(len(token))    
        pad_targets_lengths.append(len(token) + max_target_length)
    model = build_graph(keep_probability, rnn_size, num_layers, len(phrases), 
                                learning_rate, embedding_size, direction, char_to_int_letters)
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, checkpoint)
        
        output_logits = sess.run(model.predictions, {model.inputs: pad_tokens,
                                                    model.inputs_length: pad_tokens_lengths,
                                                    model.targets_length: [max_target_length],
                                                    model.keep_prob: 1.0})
    pad = char_to_int_rest['<PAD>']
    return ["".join([int_to_char_letters[i] for i in output_logit if i != pad]).replace("<EOS>","").replace("<SELF>", phrases[j].split("<SEP>")[2])
            for j, output_logit in enumerate(output_logits)]


In [33]:
predictions =  score_all_letters(list(val_data_letters['before_phrase']))
res = pd.DataFrame(zip(list(val_data_letters['before_phrase']),list(val_data_letters['before']), 
                       list(val_data_letters['after']),predictions))

print(np.mean(res[2] == res[3]))
res[res[2] != res[3]]

INFO:tensorflow:Restoring parameters from ./all_letters_model_1.ckpt
0.984244879586


,0,1,2,3
126,"rograms<SEP>—<SEP>NORML<SEP>""<SEP>.",NORML,NORML,n o r m l
143,<SEP><SEP>USAR<SEP>task<SEP>forces,USAR,USAR,u s a r
166,"<SEP>""<SEP>COPYING<SEP>file<SEP>from",COPYING,COPYING,c o p y i n g
173,",<SEP>the<SEP>TULF<SEP>remained<SEP>he",TULF,t u l f,TULF
207,State<SEP>bill<SEP>AB<SEP>1519<SEP>(,AB,a b,AB
268,<SEP><SEP>WISDOM<SEP>Television<SEP>,WISDOM,WISDOM,w i s d o m
353,<SEP><SEP>SHODAN<SEP>:<SEP>True,SHODAN,s h o d a n,SHODAN
379,unit<SEP>and<SEP>ALU<SEP>into<SEP>what,ALU,ALU,a l u
426,"<SEP><SEP>PL<SEP>360<SEP>,",PL,p l,place
490,<SEP><SEP>AAG<SEP>rocomputer<SEP>,AAG,AAG,a a g


In [143]:
epochs = 100
batch_size = 400
num_layers = 3
rnn_size = 128
embedding_size = len(chars_rest)
learning_rate = 0.0004
direction = 2
keep_probability = 0.75

log_string = 'all_rest_model_1'
model_letters = build_graph(keep_probability, rnn_size, num_layers, batch_size, 
                    learning_rate, embedding_size, direction, char_to_int_rest)
train(model_letters, epochs, log_string, int_before_dev_rest, int_after_dev_rest, 
      int_before_val_rest, int_after_val_rest, char_to_int_rest, reset = False)

INFO:tensorflow:Restoring parameters from ./all_rest_model_1.ckpt
()
Training Model: all_rest_model_1
Epoch   1/100 Batch   50/653 - Loss:  0.137, Seconds: 55.41
Epoch   1/100 Batch  100/653 - Loss:  0.003, Seconds: 55.18
Epoch   1/100 Batch  150/653 - Loss:  0.002, Seconds: 55.72
Epoch   1/100 Batch  200/653 - Loss:  0.002, Seconds: 55.01
Epoch   1/100 Batch  250/653 - Loss:  0.002, Seconds: 57.55
Epoch   1/100 Batch  300/653 - Loss:  0.002, Seconds: 52.47
Epoch   1/100 Batch  350/653 - Loss:  0.002, Seconds: 60.02
Epoch   1/100 Batch  400/653 - Loss:  0.002, Seconds: 57.79
Epoch   1/100 Batch  450/653 - Loss:  0.002, Seconds: 61.68
Epoch   1/100 Batch  500/653 - Loss:  0.002, Seconds: 55.10
Epoch   1/100 Batch  550/653 - Loss:  0.002, Seconds: 62.72
Epoch   1/100 Batch  600/653 - Loss:  0.002, Seconds: 56.18
Epoch   1/100 Batch  650/653 - Loss:  0.002, Seconds: 57.15
Testing Loss:  0.002, Seconds: 87.59
New Record!
Epoch   2/100 Batch   50/653 - Loss:  0.084, Seconds: 60.10
Epoch   2

KeyboardInterrupt: 

In [128]:
epochs = 100
batch_size = 300
num_layers = 3
rnn_size = 128
embedding_size = len(chars_rest)
learning_rate = 0.0005
direction = 2
keep_probability = 0.75

def token_to_ints_rest(token):
    return [char_to_int_rest[c] for c in token if c in chars_rest]


def score_all_rest(phrases, max_target_length = 150):
    checkpoint = "./all_rest_model_1.ckpt"
    epochs = 100
    batch_size = 500
    num_layers = 3
    rnn_size = 128
    embedding_size = len(chars_rest)
    learning_rate = 0.0005
    direction = 2
    keep_probability = 0.75
#     tokens1 = [token_to_ints(prev_token) + [char_to_int['<SEP>']] + token_to_ints(token) 
#                for prev_token, token in zip(prev_tokens, tokens)] 
    tokens1 = [reduce(lambda a,b : a + b, [token_to_ints_rest(p) + [char_to_int_rest['<SEP>']] 
                                           for p in phrase.split("<SEP>")])[:-1] for phrase in phrases] 
    
    pad_tokens = pad_sentence_batch(tokens1, char_to_int_rest)
        
        # Need the lengths for the _lengths parameters
    pad_tokens_lengths = []
    pad_targets_lengths = []
    for token in pad_tokens:
        pad_tokens_lengths.append(len(token))    
        pad_targets_lengths.append(len(token) + max_target_length)
    model = build_graph(keep_probability, rnn_size, num_layers, len(phrases), 
                                learning_rate, embedding_size, direction, char_to_int_rest)
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, checkpoint)
        
        output_logits = sess.run(model.predictions, {model.inputs: pad_tokens,
                                                    model.inputs_length: pad_tokens_lengths,
                                                    model.targets_length: [max_target_length],
                                                    model.keep_prob: 1.0})
    pad = char_to_int_rest['<PAD>']
    return ["".join([int_to_char_rest[i] for i in output_logit if i != pad]).replace("<EOS>","").replace("<SELF>", phrases[j].split("<SEP>")[2])
            for j, output_logit in enumerate(output_logits)]


In [144]:
predictions =  score_all_rest(list(val_rest['before_phrase']))
res = pd.DataFrame(zip(list(val_rest['before_phrase']),list(val_rest['before']), 
                       list(val_rest['after']),predictions))

print(np.mean(res[2] == res[3]))
res[res[2] != res[3]]

INFO:tensorflow:Restoring parameters from ./all_rest_model_1.ckpt
0.992082641046


,0,1,2,3
43,"passports<SEP>,<SEP>id's<SEP>,<SEP>ipods",id's,id's,i d's
60,"6<SEP>Ferdinand<SEP>I<SEP>,<SEP>Holy",I,the first,I
147,"<SEP>oncaptivos<SEP>meos<SEP>,<SEP>qui",meos,m e o s,meos
354,<SEP>Nacionalna<SEP>ili<SEP>etnicka<SEP>ost,ili,i l i,ili
366,the<SEP>protein<SEP>junctophilin-<SEP>3<SEP>.,junctophilin-,j u n c t o p h i l i n,j u n c t o p p i l i n
441,<SEP>2003<SEP>-<SEP>2004<SEP>:,-,-,to
446,illiliter<SEP>(<SEP>mIU<SEP>/<SEP>ml,mIU,mIU,m i u
480,nal<SEP>Fantasy<SEP>IV's<SEP>Kain<SEP>ghwind,IV's,four's,the fourth's
481,nal<SEP>Fantasy<SEP>XIII's<SEP>Lightning<SEP>.,XIII's,thirteen's,the eigthenth's
576,nvented<SEP>the<SEP>vaporised<SEP>oil<SEP>burner,vaporised,vaporised,vaporized


In [133]:
np.mean(res[res[1] == ":"][2] == res[res[1] == ":"][3])

0.98249961282329257

In [20]:
epochs = 100
batch_size = 600
num_layers = 3
rnn_size = 128
embedding_size = len(chars_proper_nouns)
learning_rate = 0.0006
direction = 2
keep_probability = 0.75

log_string = 'all_proper_nouns_model_1'
model_proper_nouns = build_graph(keep_probability, rnn_size, num_layers, batch_size, 
                    learning_rate, embedding_size, direction, char_to_int_proper_nouns)
train(model_proper_nouns, epochs, log_string, int_before_dev_proper_nouns, int_after_dev_proper_nouns, 
      int_before_val_proper_nouns, int_after_val_proper_nouns, char_to_int_proper_nouns, reset = False)

INFO:tensorflow:Restoring parameters from ./all_proper_nouns_model_1.ckpt
()
Training Model: all_proper_nouns_model_1
Epoch   1/100 Batch   50/821 - Loss:  1.234, Seconds: 72.85
Epoch   1/100 Batch  100/821 - Loss:  0.004, Seconds: 76.54
Epoch   1/100 Batch  150/821 - Loss:  0.004, Seconds: 71.14
Epoch   1/100 Batch  200/821 - Loss:  0.003, Seconds: 68.99
Epoch   1/100 Batch  250/821 - Loss:  0.003, Seconds: 74.87
Epoch   1/100 Batch  300/821 - Loss:  0.002, Seconds: 92.74
Epoch   1/100 Batch  350/821 - Loss:  0.002, Seconds: 77.37
Epoch   1/100 Batch  400/821 - Loss:  0.002, Seconds: 78.97
Epoch   1/100 Batch  450/821 - Loss:  0.002, Seconds: 73.31
Epoch   1/100 Batch  500/821 - Loss:  0.002, Seconds: 73.01
Epoch   1/100 Batch  550/821 - Loss:  0.002, Seconds: 81.09
Epoch   1/100 Batch  600/821 - Loss:  0.002, Seconds: 76.37
Epoch   1/100 Batch  650/821 - Loss:  0.002, Seconds: 74.63
Epoch   1/100 Batch  700/821 - Loss:  0.002, Seconds: 75.42
Epoch   1/100 Batch  750/821 - Loss:  0.00

KeyboardInterrupt: 

In [21]:
def token_to_ints_proper_nouns(token):
    return [char_to_int_proper_nouns[c] for c in token if c in chars_proper_nouns]


def score_all_proper_nouns(phrases, max_target_length = 150):
    checkpoint = "./all_proper_nouns_model_1.ckpt"
    epochs = 100
    batch_size = 500
    num_layers = 3
    rnn_size = 128
    embedding_size = len(chars_proper_nouns)
    learning_rate = 0.0005
    direction = 2
    keep_probability = 0.75
#     tokens1 = [token_to_ints(prev_token) + [char_to_int['<SEP>']] + token_to_ints(token) 
#                for prev_token, token in zip(prev_tokens, tokens)] 
    tokens1 = [reduce(lambda a,b : a + b, [token_to_ints_proper_nouns(p) + [char_to_int_proper_nouns['<SEP>']] 
                                           for p in phrase.split("<SEP>")])[:-1] for phrase in phrases] 
    
    pad_tokens = pad_sentence_batch(tokens1, char_to_int_proper_nouns)
        
        # Need the lengths for the _lengths parameters
    pad_tokens_lengths = []
    pad_targets_lengths = []
    for token in pad_tokens:
        pad_tokens_lengths.append(len(token))    
        pad_targets_lengths.append(len(token) + max_target_length)
    model = build_graph(keep_probability, rnn_size, num_layers, len(phrases), 
                                learning_rate, embedding_size, direction, char_to_int_proper_nouns)
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, checkpoint)
        
        output_logits = sess.run(model.predictions, {model.inputs: pad_tokens,
                                                    model.inputs_length: pad_tokens_lengths,
                                                    model.targets_length: [max_target_length],
                                                    model.keep_prob: 1.0})
    pad = char_to_int_proper_nouns['<PAD>']
    return ["".join([int_to_char_proper_nouns[i] for i in output_logit if i != pad]).replace("<EOS>","").replace("<SELF>", phrases[j].split("<SEP>")[2])
            for j, output_logit in enumerate(output_logits)]



In [22]:
n_pred = 10000
predictions =  score_all_proper_nouns(list(val_proper_nouns['before_phrase'])[:n_pred])
res = pd.DataFrame(zip(list(val_proper_nouns['before_phrase'])[:n_pred],list(val_proper_nouns['before'])[:n_pred], 
                       list(val_proper_nouns['after'])[:n_pred],predictions))

print(np.mean(res[2] == res[3]))
res[res[2] != res[3]]

INFO:tensorflow:Restoring parameters from ./all_proper_nouns_model_1.ckpt
0.9941


,0,1,2,3
23,"Bulgaria<SEP>)<SEP>Boev<SEP>,<SEP>Z.",Boev,b o e v,Boev
173,and<SEP>Mynydd<SEP>Tarw<SEP>.<SEP>,Tarw,t a r w,Tarw
750,<SEP><SEP>Knai<SEP>Thoma<SEP>and,Knai,k n a i,Knai
875,"Primiero<SEP>,<SEP>Imer<SEP>,<SEP>Mezzano",Imer,i m e r,Imer
941,"holidays<SEP>""<SEP>Revitalise<SEP>.<SEP>",Revitalise,revitalize,Revitalise
1166,"<SEP><SEP>Agmo<SEP>2007<SEP>,",Agmo,a g m o,Agmo
1524,<SEP>De<SEP>Asse<SEP>et<SEP>partibus,Asse,a s s e,Asse
1547,"files<SEP>for<SEP>SoS<SEP>""<SEP>.",SoS,s o s,SoS
1553,",<SEP>52, <SEP>No<SEP>3,<SEP>,",No,number,No
1577,kings<SEP>owner<SEP>Zygi<SEP>Wilf<SEP>potted,Zygi,Zygi,z y g i


In [23]:
np.mean(res[res[1] == "No"][2] == res[res[1] == "No"][3])

0.91176470588235292

In [25]:
res[res[1] == "No"][3].value_counts()

No        32
number     2
Name: 3, dtype: int64

In [ ]:
test_all[test_all.before == "&"][['before', 'after', 'sentence_id']].groupby(['before', 'after']).count() 

In [ ]:
train_data[train_data.before.map(lambda x : x.lower()) == "centre"][['before', 'after', 'sentence_id']].groupby(['before', 'after']).count()

In [126]:
clean_map = {"&" : "and", "centre" : "center", "#":"number", "theatre" : "theater", "i.e." : "i e", "metre" : "meter",
            }
np.mean(val_clean.after == val_clean.before.map(lambda x : clean_map[x.lower()] if x.lower() in clean_map.keys() else 
                                                x[:-3].lower() + "or" if x[-3:] == "our" and len(x) > 4 else 
                                                x[:-5].lower() + "ored" if x[-5:] == "oured" and len(x) > 6 else x)) 

0.99937467655683976

In [ ]:
val_clean[(val_clean.before.map(lambda x : clean_map[x.lower()] if x.lower() in clean_map.keys() else 
                                x[:-3].lower() + "or" if x[-3:] == "our" and len(x) > 4 else 
                                x[:-5].lower() + "ored" if x[-5:] == "oured" and len(x) > 6 else x)
           != val_clean.after)][['before', 'after', 'sentence_id']].groupby(
                                    ['before', 'after']).count().sort_values(by = 'sentence_id', ascending = False)

In [ ]:
val_proper_nouns[val_proper_nouns.before != val_proper_nouns.after][['before', 'after', 'sentence_id']].groupby(
                                    ['before', 'after']).count().sort_values(by = 'sentence_id', ascending = False)

In [145]:
(val_punct.shape[0]*1.0 + val_clean.shape[0]*0.99937467655683976 + val_proper_nouns.shape[0]*0.9941 + 
 val_elec.shape[0]*0.792315680166 + val_rest.shape[0]*0.992082641046 + val_data_numeric.shape[0]*0.992 + 
 val_data_letters.shape[0]*0.984244879586)/(val_punct.shape[0] + val_clean.shape[0] + val_proper_nouns.shape[0] + 
 val_elec.shape[0] + val_rest.shape[0] + val_data_numeric.shape[0] + val_data_letters.shape[0])

0.998197298333551

In [27]:
test_data['prev_before'] = test_data['before'].shift(1)
test_data['prev_before'][test_data.sentence_id != test_data.sentence_id.shift(1)] = ""
test_data['prev_prev_before'] = test_data['before'].shift(2)
test_data['prev_prev_before'][test_data.sentence_id != test_data.sentence_id.shift(2)] = ""
test_data['next_before'] = test_data['before'].shift(-1)
test_data['next_before'][test_data.sentence_id != test_data.sentence_id.shift(-1)] = ""
test_data['next_next_before'] = test_data['before'].shift(-2)
test_data['next_next_before'][test_data.sentence_id != test_data.sentence_id.shift(-2)] = ""

test_data['before_phrase'] = (test_data.prev_prev_before + "<SEP>" + test_data.prev_before + "<SEP>" +  test_data.before 
                                + "<SEP>" + test_data.next_before + "<SEP>" + test_data.next_next_before)

test_data['before_phrase'] = test_data['before_phrase'].map(truncate)

/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launche

In [146]:
test_data_numeric = test_data[(test_data.before.map(lambda x : max([i in x for i in "0123456789"])) == 1) & 
                       (test_data.before.map(lambda x : 1.0 if len(rx.findall(x)) > 0 and 
                        " " not in x and len(x) > 5 else 0.0) == 0.0)]
test_data_letters = test_data[(test_data.index.isin(test_data_numeric.index) == False) & 
                ((test_data.before.map(lambda x : x.upper() == x)) | 
                 (test_data.before.map(lambda x : max([i in "aeiouy" for i in x.lower()])) == 0)) &
                (test_data.before.map(lambda x : min([i in "abcdefghijklmnopqrstuvwxyz. " for i in x.lower()])) == 1) &
                (test_data.before.map(len) > 1)]
test_rest = test_data[(test_data.index.isin(test_data_numeric.index) == False) & 
         (test_data.index.isin(test_data_letters.index) == False)]
test_punct = test_rest[test_rest.before.isin(punctuations)]
test_rest = test_rest[test_rest.before.isin(punctuations) == False]

test_clean = test_rest[test_rest.before.map(lambda x : x.lower() in clean_words)]
test_rest = test_rest[test_rest.before.map(lambda x : x.lower() in clean_words) == False]

test_elec = test_rest[test_rest.before.map(lambda x : len(rel.findall(x)) > 0)]
test_rest = test_rest[test_rest.before.map(lambda x : len(rel.findall(x)) == 0)]

test_proper_nouns = test_rest[test_rest.before.map(lambda x : len(rpn.findall(x)) > 0)]
test_rest = test_rest[test_rest.before.map(lambda x : len(rpn.findall(x)) == 0)]

test_data_numeric.shape, test_data_letters.shape, test_punct.shape, test_clean.shape, test_proper_nouns.shape, test_elec.shape, test_rest.shape

((45740, 8),
 (19937, 8),
 (186826, 8),
 (595016, 8),
 (69555, 8),
 (674, 8),
 (38298, 8))

In [147]:
test_data_numeric['after'] = score_all_numeric(list(test_data_numeric['before_phrase']))
test_data_letters['after'] = score_all_letters(list(test_data_letters['before_phrase']))
test_rest1 = test_rest[test_rest.before.map(len) < 80]
test_rest1['after'] = score_all_rest(list(test_rest1['before_phrase']))
test_rest2 = test_rest[test_rest.before.map(len) >= 80]
test_rest2['after'] = test_rest2.before
test_clean['after'] = test_clean.before
test_clean.after = test_clean.before.map(lambda x : clean_map[x.lower()] if x.lower() in clean_map.keys() else 
                                         x[:-3].lower() + "or" if x[-3:] == "our" and len(x) > 4 else 
                                x[:-5].lower() + "ored" if x[-5:] == "oured" and len(x) > 6 else x)
test_punct['after'] = test_punct.before
test_proper_nouns['after'] = score_all_proper_nouns(list(test_proper_nouns['before_phrase']))
test_elec['after'] = test_elec.before.map(clean_elec)

INFO:tensorflow:Restoring parameters from ./all_numeric_model_1.ckpt


/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


INFO:tensorflow:Restoring parameters from ./all_letters_model_1.ckpt


/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


INFO:tensorflow:Restoring parameters from ./all_rest_model_1.ckpt


/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/sourabhjha/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


INFO:tensorflow:Restoring parameters from ./all_proper_nouns_model_1.ckpt


In [148]:
test_all = test_data_numeric.append(test_data_letters).append(test_rest1).append(test_rest2).append(test_clean).append(
    test_punct).append(test_proper_nouns).append(test_elec)

In [149]:
test_all['id'] = test_all.apply(lambda row : str(row['sentence_id']) + "_" + str(row['token_id']), axis = 1)
test_all[['id', 'after']].to_csv("./seq2seq_submission.csv", index = False)

In [150]:
test_all.shape, sample_submission.shape

((956046, 10), (956046, 2))